In [36]:
import pyspark
import os
import sys
from pyspark import SparkContext
from pyspark import SparkConf
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
from pyspark.sql import SparkSession

In [37]:
spark = SparkSession.builder.config("spark.driver.memory", "4g").appName('square').getOrCreate()

In [38]:
import pandas as pd
from pyspark.sql import functions as F
df = spark.createDataFrame([
(1, "Anubhab"),
(2, "Bobby"),
(3, "Carrot"),
(4, "Ded"),
(5, "Eeeewwww")
], ["roll", "name"])
print("Count",df.count())
df.show()
df.printSchema() # It will print the Schema
df = df.filter(df["name"].startswith("A"))
df = df.withColumn("marks", df["roll"] * 10)
df.write.csv('DTA.csv')
df.show()
df.show()

Count 5
+----+--------+
|roll|    name|
+----+--------+
|   1| Anubhab|
|   2|   Bobby|
|   3|  Carrot|
|   4|     Ded|
|   5|Eeeewwww|
+----+--------+

root
 |-- roll: long (nullable = true)
 |-- name: string (nullable = true)

+----+-------+-----+
|roll|   name|marks|
+----+-------+-----+
|   1|Anubhab|   10|
+----+-------+-----+

+----+-------+-----+
|roll|   name|marks|
+----+-------+-----+
|   1|Anubhab|   10|
+----+-------+-----+



In [39]:
import pandas as pd
from pyspark.sql import functions as F
df_pd = pd.DataFrame(
data={'integers': [1, 2, 3],
'floats': [-1.0, 0.5, 2.7],
'integer_arrays': [[1, 2], [3, 4, 5], [6, 7, 8, 9]]}
)
df = spark.createDataFrame(df_pd)
df.printSchema() # It will print the Schema
from pyspark.sql.functions import col, avg, sum, min, max

df.agg(sum("integers")).show()
df.agg(avg("integers")).show()
df.select(avg(col("integers"))).show()
df.agg(min("integers")).show()
df.agg(max("integers")).show()
df.show()

root
 |-- integers: long (nullable = true)
 |-- floats: double (nullable = true)
 |-- integer_arrays: array (nullable = true)
 |    |-- element: long (containsNull = true)

+-------------+
|sum(integers)|
+-------------+
|            6|
+-------------+

+-------------+
|avg(integers)|
+-------------+
|          2.0|
+-------------+

+-------------+
|avg(integers)|
+-------------+
|          2.0|
+-------------+

+-------------+
|min(integers)|
+-------------+
|            1|
+-------------+

+-------------+
|max(integers)|
+-------------+
|            3|
+-------------+

+--------+------+--------------+
|integers|floats|integer_arrays|
+--------+------+--------------+
|       1|  -1.0|        [1, 2]|
|       2|   0.5|     [3, 4, 5]|
|       3|   2.7|  [6, 7, 8, 9]|
+--------+------+--------------+



In [40]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
# Load the text file
lines = sc.textFile("sentence.txt")
counts = lines.flatMap(lambda line: line.split(" ")) \
.map(lambda word: (word, 1)) \
.reduceByKey(lambda x, y: x + y)
output = counts.collect()
for (word, count) in output:
    print("%s: %i" % (word, count))
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
spark = SparkSession.builder.getOrCreate()
lines = spark.read.text("sentence.txt")
words = lines.withColumn('word', f.explode(f.split(f.col('value'), ' '))).groupBy('word').count()\
.sort('count', ascending=False)\
.show()
spark.stop()

hule: 4
hula: 1
re: 1
wakkawoooooo: 1
+------------+-----+
|        word|count|
+------------+-----+
|        hule|    4|
|        hula|    1|
|          re|    1|
|wakkawoooooo|    1|
+------------+-----+

